In [1]:
import os
import joblib

In [2]:
# How to load our model in
loaded_model = joblib.load("./random_forest_new.joblib")

In [4]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show

In [5]:
x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), width=400, height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

amp = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
freq = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
phase = Slider(start=-6.4, end=6.4, value=0, step=.1, title="Phase")
offset = Slider(start=-9, end=9, value=0, step=.1, title="Offset")

callback = CustomJS(args=dict(source=source, amp=amp, freq=freq, phase=phase, offset=offset),
                    code="""
    const A = amp.value
    const k = freq.value
    const phi = phase.value
    const B = offset.value

    const x = source.data.x
    const y = Array.from(x, (x) => B + A*Math.sin(k*x+phi))
    source.data = { x, y }
""")

amp.js_on_change('value', callback)
freq.js_on_change('value', callback)
phase.js_on_change('value', callback)
offset.js_on_change('value', callback)

show(row(plot, column(amp, freq, phase, offset)))

In [ ]:
# Have this create a line plot where there is one point for each EF rating given the other parameters.
# Thus, the user would not have to guess the tornado strength, this would give a range of values.
# Could just give a few example counties to make things easier. Would have drop downs 
# for county, day/night, and season 

In [ ]:
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure, show
from datetime import timedelta
#from bokeh.io import output_notebook

import pandas as pd
from datetime import datetime
import numpy as np

#output_notebook()

def get_data(station_id, year):
    ''' Retrieves data from GHCN using the supplied station ID and year. 
    Returns a ColumnDataSource dataframe with record max/min, average max/min,
    and max/min for given year as well as left/right bounds for plotting later.
    '''

    df = pd.DataFrame(columns=[])
    df['date'] = daily_min_year.index
    df['actual_min'] = daily_min_year.values
    df['actual_max'] = daily_max_year.values
    df['avg_min'] = daily_min_average.values
    df['avg_max'] = daily_max_average.values
    df['record_min'] = daily_min_records.values
    df['record_max'] = daily_max_records.values
    df['left'] = df.date - timedelta(days=0.5)
    df['right'] = df.date + timedelta(days=0.5)

    test = pd.DataFrame(columns=['EF_Rate', 'EP_MOBILE', 
                        'Density', 'DJF', 'MAM', 'JJA', 'SON', 'NIGHT'])

    out = loaded_model.predict(test)

    return ColumnDataSource(data=df)

def make_plot(source, title):
    ''' Creates Bokeh plot object '''
    plot = figure(x_axis_type="datetime", width=800, tools="", 
                    toolbar_location=None)
    plot.title.text = title
    plot.line(x, y, linewidth=3)
    # fixed attributes
    plot.xaxis.axis_label = None
    plot.yaxis.axis_label = "Temperature (C)"
    plot.axis.axis_label_text_font_style = "bold"
    #plot.x_range = DataRange1d(range_padding=0.0)
    plot.grid.grid_line_alpha = 0.3

    return plot

def update_plot(attrname, old, new):
    ''' Updates Bokehplot when inputs are changed '''
    city = city_select.value
    year = year_select.value
    plot.title.text = ("Weather data for " + cities[city]['title'] 
                        + ' ' + str(year))
    src = get_data(cities[city]['station_id'], year)
    source.data.update(src.data)

# City and year to display at start
city = 'Dayton'
year = '2010'

# List of available cities and station IDs
cities = {
    'Champaign': {
        'station_id': 'USC00118740',
        'title': 'Champaign, IL',
    },
    'Dayton': {
        'station_id': 'USW00093815',
        'title': 'Dayton, OH',
    },
    'Columbus': {
        'station_id': 'USW00014821',
        'title': 'Columbus, OH',
    },
    'Chicago': {
        'station_id': 'USC00111577',
        'title': 'Chicago, IL',
    },
    'Atlanta': {
        'station_id': 'USW00013874',
        'title': 'Atlanta, GA',
    },
}

# Create list of years to select from
years_int = np.arange(1981, 2023, 1)
years = [str(x) for x in years_int]
# Create selections for dropdowns
city_select = Select(value=city, title='City', options=sorted(cities.keys()))
year_select = Select(value=year, title='Year', options=years)
# Call functions to retrieve data and make plots on start-up
source = get_data(cities[city]['station_id'], year)
plot = make_plot(source, "Weather data for " + cities[city]['title'] 
                + ' ' + str(year))
# Call update function when selections are changed
city_select.on_change('value', update_plot)
year_select.on_change('value', update_plot)
# Create dropdown menus
controls = column(city_select, year_select)

curdoc().add_root(row(plot, controls))
curdoc().title = "Weather"